In [52]:
import re
import pandas as pd
import nltk

from nltk.tree import Tree
# This uses corenlp server! Will need to alter code if using JAR files directly
# https://stanfordnlp.github.io/CoreNLP/corenlp-server.html
from nltk.parse.corenlp import CoreNLPParser
from nltk.tag.stanford import CoreNLPTagger, CoreNLPPOSTagger, CoreNLPNERTagger

import numpy as np
from nltk.corpus import wordnet as wn
from nltk.metrics import edit_distance as ed

essay_key = pd.read_csv('D:/UIC/Spring 2018/NLP/Project/essay_split.csv', sep=',')
essay_key.head()

,filename,grade,word_len,grader
0,990384.txt,high,568,Aldo
1,395987.txt,high,508,John
2,1949465.txt,high,458,Aldo
3,38209.txt,high,456,John
4,1834502.txt,high,454,Aldo


In [53]:
essays = []
for filename in essay_key['filename']:
    with open('D:/UIC/Spring 2018/NLP/Project/essays_dataset/essays/'+filename, 'r') as f:
        essays.append(f.read().strip())
        
essay_key['essay'] = essays
essay_key.head()

,filename,grade,word_len,grader,essay
0,990384.txt,high,568,Aldo,I dont think that in twenty years ther will be...
1,395987.txt,high,508,John,I disagree with the statement. Lets guess what...
2,1949465.txt,high,458,Aldo,I agree that twenty from now there will be few...
3,38209.txt,high,456,John,It is generally believed that young people enj...
4,1834502.txt,high,454,Aldo,"There is a famous saying which reads: ""Give a ..."


In [54]:
matches = wn.synsets('Dog')
len(matches)

8

In [55]:
def pos_tags(tagger, sentences):
    """Tags sentences with POS tags. Returns a list of (word, tag) tuples

    tagger (CoreNLPTagger): a tagger to tag sentences
    RETURNS (list): list of (word, tag) tuples
    """
    default_properties = {'annotators': 'tokenize,ssplit,pos'}
    tagged_data = tagger.api_call(sentences, properties=default_properties)
    
    tags = list()
    for sent in tagged_data['sentences']:
        #modified to return the start index and end index of the word
        tags.append([(token['word'], token['pos'], token['characterOffsetBegin'], token['characterOffsetEnd']) for token in sent['tokens']])
    return tags

In [56]:
tagger = CoreNLPPOSTagger(url='http://localhost:9000')

In [57]:
result = pos_tags(tagger, essay_key.loc[0,'essay'])
result

[[('I', 'PRP', 0, 1),
  ('dont', 'VBP', 2, 6),
  ('think', 'VB', 7, 12),
  ('that', 'IN', 13, 17),
  ('in', 'IN', 18, 20),
  ('twenty', 'CD', 21, 27),
  ('years', 'NNS', 28, 33),
  ('ther', 'RB', 34, 38),
  ('will', 'MD', 39, 43),
  ('be', 'VB', 44, 46),
  ('fewer', 'JJR', 47, 52),
  ('cars', 'NNS', 53, 57),
  ('than', 'IN', 58, 62),
  ('now', 'RB', 63, 66),
  ('because', 'IN', 67, 74),
  ('sales', 'NNS', 75, 80),
  ('of', 'IN', 81, 83),
  ('cars', 'NNS', 84, 88),
  ('are', 'VBP', 89, 92),
  ('increasing', 'VBG', 93, 103),
  ('day', 'NN', 104, 107),
  ('by', 'IN', 108, 110),
  ('day', 'NN', 111, 114),
  ('.', '.', 114, 115)],
 [('More', 'JJR', 116, 120),
  ('people', 'NNS', 121, 127),
  ('are', 'VBP', 128, 131),
  ('using', 'VBG', 132, 137),
  ('cars', 'NNS', 138, 142),
  ('as', 'IN', 143, 145),
  ('it', 'PRP', 146, 148),
  ('is', 'VBZ', 149, 151),
  ('easy', 'JJ', 152, 156),
  ('to', 'TO', 157, 159),
  ('own', 'VB', 160, 163),
  ('a', 'DT', 164, 165),
  ('car', 'NN', 166, 169),
  ('to

In [58]:
#https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

In [59]:
for sent in result:
    for pos_t in sent:
        matches = wn.synsets(pos_t[0])
        
        print(pos_t[0], pos_t[1], get_wordnet_pos(pos_t[1]),len(matches))

I PRP  4
dont VBP v 0
think VB v 14
that IN  0
in IN  7
twenty CD  3
years NNS n 7
ther RB r 0
will MD  6
be VB v 14
fewer JJR a 2
cars NNS n 5
than IN  0
now RB r 8
because IN  0
sales NNS n 6
of IN  0
cars NNS n 5
are VBP v 14
increasing VBG v 4
day NN n 10
by IN  2
day NN n 10
. .  0
More JJR a 5
people NNS n 6
are VBP v 14
using VBG v 7
cars NNS n 5
as IN  10
it PRP  1
is VBZ v 13
easy JJ a 15
to TO  0
own VB v 2
a DT  7
car NN n 5
today NN n 4
and CC  0
many JJ a 1
low JJ a 16
cost NN n 5
cars NNS n 5
are VBP v 14
coming VBG v 26
into IN  0
the DT  0
depending VBG v 2
on IN  5
demand NN n 11
of IN  0
people NNS n 6
ans NNS n 2
usage NN n 3
. .  0
Twenty CD  3
years NNS n 7
from IN  0
now RB r 8
there EX  4
will MD  6
be VB v 14
more RBR r 5
and CC  0
more RBR r 5
people NNS n 6
coming VBG v 26
to TO  0
cities NNS n 3
and CC  0
towns NNS n 4
in IN  7
search NN n 9
of IN  0
jobs NNS n 17
and CC  0
other JJ a 4
requirements NNS n 3
and CC  0
population NN n 5
is VBZ v 13
increasing V

In [51]:
wn.synsets('that')

[]

In [61]:
from enchant.checker import SpellChecker

text = "This is sme text with a fw speling errors in it."

chkr = SpellChecker("en_US", text)
for err in chkr:
    print(err.word + " at position " + str(err.wordpos))
    err.replace("SPAM")

t = chkr.get_text()
print("\n" + t)

sme at position 8
fw at position 25
speling at position 30

This is SPAM text with a SPAM SPAM errors in it.


In [62]:
chkr = SpellChecker("en_US", essay_key.loc[0, 'essay'])
chkr

In [68]:
import enchant
dicc = enchant.Dict("en_US")
wrong_words = []
for sent in result:
    for pos_t in sent:
        word = pos_t[0]
        if dicc.check(word) is False:
            wrong_words.append(word)
            
wrong_words

['dont',
 'ther',
 'becuase',
 'becuase',
 'donot',
 'dont',
 'dont',
 ',',
 'increains',
 'increaing',
 ',',
 'devloped',
 "'s",
 'eard',
 'mediun',
 ',',
 'donot',
 'disturbace',
 'dont',
 'varities']